<br>

# Introdução

Por meio do [_site_ da Defesa Civil](http://www.defesacivil.sp.gov.br/) (mais específicamente nesse [_link_](http://www.defesacivil.sp.gov.br/coordenadores-regionais-de-defesa-civil/)) foi possível obter diversas informações sobre os Coordenadores que atuam nas 'Coordenadorias Regionais de Proteção e Defesa Civil - REPDEC'.


In [ ]:
import os
import random
import re
import time

import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from osgeo import gdal, osr
from tqdm.notebook import tqdm, trange

from sp_defesacivil_divadmin.my_paths import output_path_tabs

<br>

# Dados Tabulares


## Scrapy _Site_

Iniciamente foi obtido o código fonte da paǵina, usando o _BeautifulSoup_.


In [ ]:
r = requests.get('http://www.defesacivil.sp.gov.br/coordenadores-regionais-de-defesa-civil/')
soup = BeautifulSoup(r.content, 'html.parser')
content_tag = soup.find('div', {'id': 'content'})


In [ ]:
list_text = []
for i in content_tag.find_all('p'):
    print(i.text)
    list_text.append(i.text.replace('\xa0', ' '))
list_text

<br>

## Ajuste da tabela

A partir disso, montei uma lista com todos os valores que estavam na página e converti a lista em uma tabela. Adicionei uma coluna contendo o valor do índice quando, na linha da tabela, houvesse a palavra "REPDEC e preenchi essa tabela.


In [ ]:
# Create Table
df = pd.DataFrame({'data': list_text})
df.head()

In [ ]:
df = df[df['data'] != ' ']
df

In [ ]:
# Define os marcos iniciais
df['repdec'] = df.apply(
    lambda x: x['data'] if 'REPDEC' in x['data'] else np.nan, axis=1
)

# Result
df.head(10)

In [ ]:
# Preenche Coluna
df['repdec'] = df['repdec'].ffill()

# Result
df.head(10)

In [ ]:
# dddd
#df = df[df['data'] != df['repdec']]

# Elimina Linhas com NaN
df = df.dropna()

# Result
df.head(10)

<br>

Criei sub-listas, de tamanhos diferentes, para cada uma desses conjuntos, por meio desse índice e converti em uma tabela.


In [ ]:
list_temps = []
for i in list(set(df['repdec'])):
    df_temp = df[df['repdec'] == i]
    list_temp = list(df_temp['data'])
    list_temps.append(list_temp)


In [ ]:
# 
n_columns = max([len(x) for x in list_temps])
n_columns_exclude_repdec = n_columns -1

# 
if n_columns_exclude_repdec % 3 == 0:
    n_coords = int(n_columns_exclude_repdec / 3)

# Results
n_coords

In [ ]:
# Cria tabela
df = pd.DataFrame(
    list_temps,
)

# Results
print(len(df.columns))
df.head()

In [ ]:
list_cols = []
list_itens = ['nome', 'mail', 'tel']

# Adiciona Primeira Coluna
list_cols.extend(['repdec'])

# Adiciona Colunas Seguintes (coord_regional)
list_cols.extend([f'coord_regional_{x}' for x in list_itens])

# Adiciona Primeira Coluna (coord_adj)
for n_coord_adj in range(8):    
    n_coord_adj = n_coord_adj+1
    list_cols.extend([f'coord_adj{n_coord_adj}_{x}' for x in list_itens])


# Results
print(len(list_cols))
list_cols


In [ ]:
# Cria tabela
df = pd.DataFrame(
    list_temps,
    columns=list_cols
)

<br>

Com a tabela bruta, passei a limpa-la, "splitando" os campos pelo caractere ":", além de outras correções.


In [ ]:
# Exceção: em uma célula há duas vezes o ":"
df['coord_adj1_tel'] = (
    df['coord_adj1_tel']
    .astype(str)
    .apply(lambda x: x.replace('Ramal:', 'Ramal '))
)

In [ ]:
# Remove tudo que vem antes do :
df = df.applymap(lambda x: x.split(':')[-1], na_action='ignore')

In [ ]:
# Divide Células em Duas Colunas
df[['repdec_nome', 'repdec_cod']] = df['repdec'].str.split('–', expand=True)
df[['del', 'repdec_cod']] = df['repdec'].str.split('/', expand=True)
df.drop(['del', 'repdec'], axis=1, inplace=True, errors='ignore')

# Results
df.head()

## Funções

Funções para renomear campos


In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '\xa0': ' ',
        '  ': ' ',
        # Basics
        ' Com ': ' com ',
        ' Sobre ': ' sobre ',
        ' Da ': ' da ',
        ' De ': ' de ',
        ' Do ': ' do ',
        ' Das ': ' das ',
        ' Dos ': ' dos ',
        ' A ': ' a ',
        ' As ': ' as ',
        ' Ao ': ' ao ',
        ' Aos ': ' aos ',
        ' E ': ' e ',
        ' O ': ' o ',
        ' Os ': ' os ',
        # Erros
        '1ºten': '1º Ten',
        # Abreviações
        'Subten ': 'Subtenente ',
        'Sub Ten': 'Subtenente ',
        'Cap ': 'Capitão ',
        'Ten ': 'Tenente ',
        'Maj ': 'Major ',
        'Cel ': 'Coronel ',
        'Sgt ': 'Sargento ',
        ' Pm ': ' PM ',
        # Empty
        'None': '',
        'none': '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
def rename_tel(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '\xa0': ' ',
        '  ': ' ',
        # Basics
        ' – ': '-',
        ')': ') ',
        ' (': ' / (',
        ' / ': '/',
        '//': '/',
        '/': ' / ',
        '  ': ' ',
        # Empty
        'None': '',
        'none': '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
def rename_mail(x):
    x = x.lower()
    x = x.strip()
    dict_rename = {
        # Encoding
        '\xa0': ' ',
        '  ': ' ',
        # Basic
        ' ': '/',
        ';': '/',
        ',': '/',
        '//': '/',
        '/': ' / ',
        '  ': ' ',
        # Empty
        'None': '',
        'none': '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

<br>

## Renomeando Campos em Colunas


In [ ]:
# Definindo lista de colunas que serão empregadas funções.
list_nome = []
list_tel = []
list_mail = []

for col in list(df.columns):
    if 'nome' in col:
        list_nome.append(col)
    if 'tel' in col:
        list_tel.append(col)
    if 'mail' in col:
        list_mail.append(col)

# Adicionando manualmente outras colunas.
list_nome.append('repdec_nome')
list_nome.append('repdec_cod')

# Aplicando funções
for field in list_nome:
    df[field] = df[field].astype(str).apply(lambda x: rename_nome(x))
for field in list_tel:
    df[field] = df[field].astype(str).apply(lambda x: rename_tel(x))
for field in list_mail:
    df[field] = df[field].astype(str).apply(lambda x: rename_mail(x))

<br>

E, por fim, reordenando colunas.


In [ ]:
# Reordena Colunas
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
df = df.reindex(cols, axis=1)

# Resultados
df.head()

## _Splita_ Telefones


In [ ]:
# Definindo lista de colunas que serão empregadas funções.
list_tel = []

for col in list(df.columns):
    if 'tel' in col:
        list_tel.append(col)

# Aplicando funções
for field in list_tel:
    tel_01 = f'{field}01'
    tel_02 = f'{field}02'
    tel_01ramal = f'{field}01ramal'
    tel_02ramal = f'{field}02ramal'

    # Split Telefone
    try:
        df[tel_01] = df[field].str.split(' / ', n=1, expand=True)[0]
    except:
        df[tel_01] = None
    try:
        df[tel_01ramal] = df[tel_01].str.split(' Ramal ', n=1, expand=True)[1]
    except:
        df[tel_01ramal] = None
    try:
        df[tel_01] = df[tel_01].str.split(' Ramal ', n=1, expand=True)[0]
    except:
        df[tel_01] = None

    try:
        df[tel_02] = df[field].str.split(' / ', n=1, expand=True)[1]
    except:
        df[tel_02] = None
    try:
        df[tel_02ramal] = df[tel_02].str.split(' Ramal ', n=1, expand=True)[1]
    except:
        df[tel_02ramal] = None
    try:
        df[tel_02] = df[tel_02].str.split(' Ramal ', n=1, expand=True)[0]
    except:
        df[tel_02] = None

    # Deleta Coluna
    df.drop(field, axis=1, inplace=True, errors='ignore')

## _Splita_ E-mails


In [ ]:
list_mail = []

for col in list(df.columns):
    if 'mail' in col:
        list_mail.append(col)

# Aplicando funções
for field in list_mail:
    mail_01 = f'{field}01'
    mail_02 = f'{field}02'

    # Split Telefone
    try:
        df[mail_01] = df[field].str.split(' / ', n=1, expand=True)[0]
    except:
        df[mail_01] = None
    try:
        df[mail_02] = df[field].str.split(' / ', n=1, expand=True)[1]
    except:
        df[mail_02] = None

    # Deleta Coluna
    df.drop(field, axis=1, inplace=True, errors='ignore')

## Reordena Colunas


In [ ]:
# Reordena Colunas
cols = df.columns.tolist()

cols.sort()
cols = cols[-2:] + cols[-9:-2] + cols[:-9]

df = df.reindex(cols, axis=1)
cols

## Salva


In [ ]:
# Results
df.to_csv(
    output_path_tabs / 'tab_defesacivil.csv',
    index=False,
)
df